## Logging and plotting data

In the previous practical sessions we saw how to define routines and attach them to the agents. Here we are going to see how to use such routines to record data perceived or produced by the agents. This will allow the plotting of figures in the notebook. This can also be useful to better visualize what is happening in your simulation and help you debugging it. 

Let's start with a simple example where we record the values returned by both proximeters through time.

First, start the simulator and create a controller object as usual (we'll use the same scene as in session 4):

In [1]:
from vivarium.controllers.notebook_controller import NotebookController
from vivarium.utils.handle_server_interface import start_server_and_interface, stop_server_and_interface

In [ ]:
start_server_and_interface(scene_name="session_4")

In [ ]:
controller = NotebookController()

In [4]:
controller.run()

Then, we will assign variables to each agent present in the scene. The cell below has the exact same effect as the one we use near the start of session 4, it is just shorter:

In [5]:
agent_0, agent_1, agent_2 = controller.agents

Now, let's add some logs to the first agent. Recording data is realized by the `add_log` method of the agent, which requires two arguments: an arbitrary label of the recorded data, what we call a *topic*, and the data to be recorded. For example:

In [7]:
# Record the number 1 in a topic called 'test' on agent_0
agent_0.add_log("test", 1)

This stores the data `1` in a topic that we arbitrarily call `"test"`. We can retrieve this data by using the `get_log` function , which requires as argument the name of the topic (`"test"` in this example):

In [ ]:
print(agent_0.get_log("test"))

Calling `agent_0.get_log("test")` returns the list of the data recorded in the topic `"test"` by `agent_0`. Here it prints `[1]`, a list containing the only data we have stored before.

Let's add another data to the same topic:

In [9]:
agent_0.add_log("test", 42)

And retrieve the data recorded on this topic:

In [ ]:
print(agent_0.get_log("test"))

The second value we have added, `42`, has been appended to the list, which now contains the two recorded data.

We can add another value to another topic:

In [11]:
agent_0.add_log("another_topic", 18)

and retrieve it using `get_log`, this time with the name of this new topic:

In [ ]:
print(agent_0.get_log("another_topic"))

Of course the data recorded before in the topic `"test"` is still accessible:

In [ ]:
print(agent_0.get_log("test"))

The names chosen for the topics are completely arbitrary. They are just labels that you choose for organizing the recorded data according to their meaning. The type of data recorded in a topic is also arbitrary: above we recorded integer values, but we could instead record strings or whatever.

These two functions allow to record various data from the simulation, organizing them by topics differentiated by their names and attaching them to specific agents. Coupled with an appropriate routine running on the agent that continuously calls the `add_log` function, for example to record the values of the proximeters or the motors through time, this can then be used for generating figures plotting what is happening in the simulation.

Let's define a routine that record the values sensed by the left and right proximeters of an agent:

In [14]:
# Define a routine using the method we have seen in the last session. Here we call it agent_log 
def agent_log(agent):
    # Retrieve the values of the left and right proximeters:
    left, right = agent.sensors()
    
    # Record the left activation in a topic called "left_prox"
    agent.add_log("left_prox", left)

    # Record the right activation in a topic called "right_prox"
    agent.add_log("right_prox", right)

Then attach it on `agent_0` as usual with the `attach_routine` function. Also set both of its motors to 1 to make it move in the simulation.

In [ ]:
# Attach the agent_log routine to agent_0, which will be executed every 50 timesteps
agent_0.attach_routine(agent_log, interval=50)

# Make the agent move forward
agent_0.left_motor = agent_0.right_motor = 1.


We set an interval of 50 for the `agent_log` routine, meaning that the left and right proximeter values will be recorded every 50 iterations in the controller loop. We could indicate a smaller interval for more precision but keep in mind that the list of recorded data could then quickly become very large, since the data are recorded continuously at the specified interval. For example, with an interval of 5 and a fps of 30, it will record $(30/5)*60=360$ values each minute of the simulation. 

You can access to the values recorded from the left proximeter with:

In [ ]:
print(agent_0.get_log("left_prox"))

We can get the length of this list, i.e. how many data has been recorded until now, with the following command:

In [ ]:
print(len(agent_0.get_log("left_prox")))

We can clear all logs by executing:

In [20]:
agent_0.clear_all_logs()

This will erase all the data that `agent_0` has recorded. Recording will however still continue to occur because the `agent_log` routine is still runnning. If you want to stop it you can detach the routine as usual:

In [22]:
agent_0.detach_routine("agent_log")

Now the data is no longer recorded.

Let's now define a population of agents foraging for resources according to their energy level (the code below is similar to the one in session 4). In addition we will define a routine continuously recording various data during the simulation, for example proximeter activations or energy levels.

First we define the obstacle avoidance behavior for all of the scene obstacles:

In [ ]:
controller.print_subtypes_list()

In [24]:
def obstacle_avoidance(agent):
    left, right = agent.sensors(sensed_entities=["s_obstacles", "b_obstacles"])
    left_wheel = 1 - right
    right_wheel = 1 - left   
    return left_wheel, right_wheel

Let's initialize the energy levels:

In [25]:
max_energy_level = 1.
init_energy_level = 0.5

for agent in controller.agents:
    agent.energy_level = init_energy_level

Define the routine computing the energy level:

In [26]:

def energy(agent): 
    if agent.has_eaten():
        # if the agent has eaten a resource since the last execution of the routine, increase its energy level
        agent.energy_level += 0.5  # This is equivalent to agent.energy_level = agent.energy_level + 0.5
    else:
        # decrease energy level
        agent.energy_level -= 0.01  # otherwise (nothing eaten), decrease the energy level a bit
    # The line below bounds the value of the energy level between 0 and max_energy_level
    agent.energy_level = min(max_energy_level, max(agent.energy_level, 0.))

Define the foraging behavior, which is weighted according to the energy level:

In [27]:
def foraging(agent):
    left, right = agent.sensors(sensed_entities=["resources"])
    left_activation = right
    right_activation = left
    return left_activation, right_activation

Define the `foraging_weight` routine that modulates the weight of the foraging behavior according to the energy level of an agent:

In [28]:
def foraging_weight(agent):
    # This routine changes the weight of the foraging behavior according to the current energy level
    # The lower the energy level, the higher the weight (energy level is bounded between 0 and 1 in the energy routine)
    # E.g., if the energy is 1 (maximum value), the behavior weight will be 0 (and vice versa)
    agent.change_behavior_weight(foraging, 1 - agent.energy_level)

And finally we define the routine that will log the data we are interested in (here the left and right activations of the proximeters and the wheels, as well as the energy level of the agent):

In [30]:
# Define a routine using the method we have seen in the last session. Here we call it agent_log 
def agent_log(agent):
    # Retrieve the values of the left and right proximeters:
    left, right = agent.sensors()
    
    # Record the left proximeter activation in the topic called "left_prox"
    agent.add_log("left_prox", left)

    # Record the right proximeter activation in the topic called "right_prox"
    agent.add_log("right_prox", right)

    # Record the energy level in the topic called "energy"
    agent.add_log("energy", agent.energy_level)

Now we can attach the behaviors and routines we have just define on all agents:

In [37]:
# First start sphere apparition in the environment:
controller.start_resources_apparition(interval=50)
controller.start_eating_mechanism(interval=30)

# For all agents
for agent in controller.agents:
    # Detach all existing behaviors and routines:
    agent.detach_all_behaviors()
    agent.detach_all_routines()

    # Set the diet of the agent to consume resources 
    agent.diet = ["resources"]

     # Attach the routines for computing the energy level, modulating the behavior weight and recording data
    agent.attach_routine(energy, interval=10)
    agent.attach_routine(foraging_weight, interval=10)
    agent.attach_routine(agent_log, interval=10)

    # Attach the two behaviors we have defined
    agent.attach_behavior(obstacle_avoidance)
    agent.attach_behavior(foraging)

   


This will start the defined behaviors and routines on the two agents. The `agent_log` routine will record the proximeter activations, as well as the energy level of each agent. Using the produced log, we can now plot those data against time. Let's for example plot the activation of the left proximeter through time. This can be done like this:

In [32]:
# Matplotlib is the standard Python library for plotting data
import matplotlib.pyplot as plt

In [ ]:
# The line below is mandatory to inform the notebook we want to plot directly in it
%matplotlib inline

# Plot the left proximeter value recorded by `agent_0`
plt.plot(agent_0.get_log("left_prox"))
plt.show()

The above figure plot all the left proximeter values recorded from `agent_0`. The x-axis corresponds to when the data was recording (e.g. at `x = 100` we have the 100th recorded value). 

We can indicate the labels of the x and y axes and provide a title for the figure with:

In [ ]:
%matplotlib inline

# Plot the left proximeter value recorded by `agent_0` and set labels to the x and y axes, as well as a title
plt.plot(agent_0.get_log("left_prox"))
plt.xlabel("Time")
plt.ylabel("Left proximeter")
plt.title("Plot of left proximeter activation against time")
plt.show()

In case the plot becomes hard to read because there are to many data (e.g. if the range on the x-axis exceeds 1000), we can clear the log. The logging will continue because the routine is still attached.

In [38]:
for agent in controller.agents:
    agent.clear_all_logs()

Let's now plot the energy level of `agent_0` (in case you cleared the plot with the cell above you might want to wait a few seconds so that new data are recorded):

In [ ]:
# The line below is mandatory to inform the notebook we want to plot directly in it
%matplotlib inline

# Plot the energy level recorded by `agent_0`
plt.plot(agent_0.get_log("energy"))
plt.xlabel("Time")
plt.ylabel("Energy level")
plt.title("Plot of energy level against time")
plt.show()

We can also plot two time series on the same plot. Let's plot the energy levels of two agents:

In [ ]:
# The line below is mandatory to inform the notebook we want to plot directly in it
%matplotlib inline

# Plot the energy levels recorded by `agent_0` and `agent_1`
plt.plot(agent_0.get_log("energy"))
plt.plot(agent_1.get_log("energy"))
plt.xlabel("Time")
plt.ylabel("Energy level")
plt.title("Plot of energy level against time")
plt.show()

We can add a legend to indicate which line corresponds to which agent:

In [ ]:
# The line below is mandatory to inform the notebook we want to plot directly in it
%matplotlib inline

# Plot the energy levels recorded by `agent_0` and `agent_1`
plt.plot(agent_0.get_log("energy"))
plt.plot(agent_1.get_log("energy"))
plt.legend(["agent_0", "agent_1"])

plt.xlabel("Time")
plt.ylabel("Energy level")
plt.title("Plot of energy level against time")
plt.show()

By right-clicking on the figure and choosing "Save image as", you can store it as a PNG image. This is useful if you want to save it for later.

That's all. Don't forget to properly close the session when you have finished with the notebook:

In [ ]:
stop_server_and_interface(safe_mode=False)
controller.stop()